In [91]:
import sys
import os
sys.path.append(os.getcwd())


import pandas as pd
import string 
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer as stemmer 
import nltk
from nltk.corpus import stopwords
from collections import Counter
import nltk.stem

# Special vocabulary module from shoyu
import vocabulary_hdp as vocab

In [69]:
stemmer = stemmer("english")

In [70]:
def preprocess(doc):
    return [stemmer.stem(WordNetLemmatizer().lemmatize(w, pos='v')) for w in doc.translate(str.maketrans('','', string.punctuation)).lower().split(' ')]

def rm_stopwords_and_short_words(words):
    results = []
    for i in words:
        if not i in stopwords1 and len(i)  > 3:
            results.append(i)
    return results

def full_preprocess(doc):
    return rm_stopwords_and_short_words(preprocess(doc))


In [71]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andrewcarr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrewcarr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
df = pd.read_csv("Downloads/tm_test_data.csv")

In [73]:
stopwords1 = stopwords.words('english')

In [74]:
tokenized_df = [full_preprocess(i) for i in df.abstract]

In [75]:
all_words = [i for sublist in tokenized_df for i in sublist]
all_words =  list(set(all_words))

In [76]:
# Filter out tokens that appear in fewer than 3 abstracts and tokens that appear in more than half the abstracts 
all_words_counts = np.zeros(len(all_words))

for k,i in enumerate(all_words):
    for j in tokenized_df:
        if i in j:
            all_words_counts[k] += 1 
            
word_counts_dict = list(zip(all_words, list(all_words_counts)))
word_counts_dict_ab = list(filter(lambda x: x[1] > 3, word_counts_dict))
word_counts_dict_ab2 = list(filter(lambda x: x[1] < len(tokenized_df)/2, word_counts_dict_ab))
final_dict = [i[0] for i in word_counts_dict_ab2]

In [77]:
tokenized_df_ab = []
for i in tokenized_df:
    tokenized_df_ab.append([j for j in i if j in final_dict])
    

In [92]:
voca = vocab.Vocabulary()
docs = [voca.doc_to_ids(doc) for doc in tokenized_df_ab]

In [93]:
# Final result - list (docs) where each element is set of words represented by numbers in a global vocabulary (final dict)
docs

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  2,
  24,
  25,
  26,
  27,
  28,
  4,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  4],
 [38,
  39,
  40,
  41,
  42,
  43,
  44,
  5,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  48,
  59,
  60,
  61,
  41,
  42],
 [0,
  62,
  63,
  64,
  65,
  66,
  4,
  44,
  67,
  68,
  69,
  70,
  4,
  71,
  66,
  8,
  17,
  5,
  72,
  66,
  73,
  74,
  75,
  76,
  77,
  13,
  78,
  41,
  79,
  49,
  80,
  81,
  66,
  82,
  41,
  44,
  35,
  83,
  84,
  41,
  85,
  86,
  87,
  66,
  17,
  6,
  88,
  89,
  90,
  91,
  44,
  92,
  93,
  94,
  95,
  44,
  92,
  62,
  96,
  97,
  66,
  73,
  98,
  41,
  28,
  92,
  99,
  100,
  101],
 [102,
  4,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  106,
  110,
  96,
  111,
  8,
  105,
  112,
  102,
  113,
  102,
  114,
  115,
  113,
  8,
  110,
  116,
  117,
  10,
  110,